In [2]:
import requests
import pandas as pd
import time

In [3]:
consumer_key = '3MVG9oZtFCVWuSwOvwwAVjUooDdD6CCxCSDG4emaWuoy.1736VDuqChc1toMhcI7qDaZ1YnmHaR3a9RAFIzFc'
consumer_secret = '6A0F3F0B19D44C059188E1302953FBE1D92DCC700DBC3DA6D4469E56E5DEA202'

auth_url = 'https://test.salesforce.com/services/oauth2'
security_token = 'aBGTznDUXb72Wncl50stlFIJ'

sf_domain = 'https://epostglobal1--devrt6.sandbox.my.salesforce.com'
opp_tree_endpoint = 'services/data/v53.0/composite/tree/Opportunity/'
composite_graph_endpoint = 'services/data/v53.0/composite/graph/'

In [4]:
username = 'ydasari@epostglobalshipping.com'
password = 'w@ckyCar33'

In [5]:
class SFSession:
    def __init__(self, username, password):
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        
        body = {
            'grant_type': 'password',
            'client_id': consumer_key,
            'client_secret': consumer_secret,
            'username': username,
            'password': password + security_token
        }
        
        self.login_response = requests.post(url=f'{auth_url}/token', headers=headers, data=body)\
                .json()
        self.token = self.login_response.get('access_token')
        
        
    def logout(self):
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        body = {'token': self.token}
        return requests.post(url=f'{auth_url}/revoke', headers=headers, data=body)
        
    def hit_endpoint(self, endpoint, body):
        headers = {'Authorization': f'Bearer {self.token}', 
                   'Content-Type':'application/json'}
        return requests.post(url=f'{sf_domain}/{endpoint}',
                             headers=headers,
                             json=body)

In [6]:
sf_session = SFSession(username, password)

In [7]:
sf_session.login_response

{'access_token': '00D3C0000005Lhg!ARkAQIJ8oZa2KYjilDgpQ_gTgBJcFPzb78fTeeYLNuz0770vaHp_Bf.8wbjwVVdmn8O2RpdWBCYnkxj_Y89tkELD3Xkd0coi',
 'instance_url': 'https://epostglobal1--devrt6.sandbox.my.salesforce.com',
 'id': 'https://test.salesforce.com/id/00D3C0000005LhgUAE/0053C000005r7fSQAQ',
 'token_type': 'Bearer',
 'issued_at': '1687895488910',
 'signature': 'x4TOjBtcL641vYuPwi0PT2xpUrq9nuk4oosQRDkUKks='}

In [10]:
test_grid = pd.read_excel('rate increase prod rrd 12-21-2022.xlsx')

In [15]:
test_grid.head(5)

,Salesperson,Roll-up,Customer,AcctNum,Service,Revenue Q1,Revenue Q2,Revenue Q3,Margin Q1,Margin Q2,...,SFProdId,AccountId,OwnerId,SalesforceOwnerName,XPOSvc,Passthrough,FinalIncrease,FinalSuggested,Upload,AutoQuote
0,Barbara Rimmer-Rausch,RRD,RRD Houston - Alpha Delta Pi,9539,Air First Class,2992.0,3015.0,3025.0,0.35,0.33,...,01t4x0000040fhyAAA,0014x00000H2Y6gAAF,0054x000004ga2fAAA,Barbara Rausch,1,False,0.050,0.06,True,True
1,Barbara Rimmer-Rausch,RRD,RRD Shakopee - US Bank,10195,02-IPA,0.0,0.0,2310.0,0.00,0.00,...,01t4x0000040fiAAAQ,0014x00001jYWSWAA4,0054x000004ga2fAAA,Barbara Rausch,19,True,0.000,0.00,True,True
2,Barbara Rimmer-Rausch,RRD,RRD-H&N Printing-Toastmasters Parcel,8561,03-Global Courier,5790.0,1753.0,360.0,0.47,0.42,...,01t4x0000040fiOAAQ,0014x00000H2W4cAAF,0054x000004ga2fAAA,Barbara Rausch,33,False,0.000,0.00,True,False
3,Barbara Rimmer-Rausch,RRD,RRD-H&N Printing-Toastmasters Parcel,8561,07-RRD Priority Parcel D/C DDU,200.0,0.0,0.0,0.86,0.00,...,01t4x0000040fjSAAQ,0014x00000H2W4cAAF,0054x000004ga2fAAA,Barbara Rausch,106,False,0.055,0.06,True,True
4,Barbara Rimmer-Rausch,RRD,RRD-H&N Printing-Toastmasters Parcel,8561,08-Global Courier,4033.0,19711.0,0.0,0.52,0.39,...,01t4x0000040fiPAAQ,0014x00000H2W4cAAF,0054x000004ga2fAAA,Barbara Rausch,33,False,0.000,0.00,True,False


In [11]:
def try_parse(x):
    try:
        return float(x)
    except:
        return 0

In [12]:
test_grid = (test_grid
                 .fillna({col:0 for col in test_grid.columns if ('Margin' in col) or ('Increase' in col) or ('Revenue' in col) or ('Pu/lb' == col)})
                 .assign(Passthrough = lambda df: df['Increase%'].map(lambda p : True if p == 'Pass-through' else False),
                         FinalIncrease = lambda df: df['Increase%'].map(try_parse),
                         FinalSuggested = lambda df: df['Suggested Increase'].map(try_parse),
                         Upload = lambda df: df['IN Salesforce'] == 1,
                         AutoQuote = lambda df: df['Auto Quote'] == 1))

In [21]:
test_grid.loc[:,'Increase%']

Salesperson                     Barbara Rimmer-Rausch
Roll-up                                           RRD
Customer                 RRD Houston - Alpha Delta Pi
AcctNum                                          9539
Service                               Air First Class
Revenue Q1                                     2992.0
Revenue Q2                                     3015.0
Revenue Q3                                     3025.0
Margin Q1                                        0.35
Margin Q2                                        0.33
Margin Q3                                        0.37
Revenue (Q1-Q3)                                  9033
Margin (Q1-Q3)                                   0.35
Margin (Q2-Q3)                                   0.35
Rollup Margin (Q2-Q3)                             0.3
Suggested Increase                               0.06
Increase%                                        0.05
Pre-Call Notes                                    NaN
Call 1 Notes                

In [18]:
test_grid.columns

Index(['Salesperson', 'Roll-up', 'Customer', 'AcctNum', 'Service',
       'Revenue Q1', 'Revenue Q2', 'Revenue Q3', 'Margin Q1', 'Margin Q2',
       'Margin Q3', 'Revenue (Q1-Q3)', 'Margin (Q1-Q3)', 'Margin (Q2-Q3)',
       'Rollup Margin (Q2-Q3)', 'Suggested Increase', 'Increase%',
       'Pre-Call Notes', 'Call 1 Notes', 'Status', 'Pu/lb', 'Facility',
       'Tariff', 'Auto Quote', 'IN Salesforce', 'Requote', 'RequoteMargin',
       'IPA Flag', 'Rank', 'SalesforceAcctNum', 'SalesforceProdName',
       'SFProdId', 'AccountId', 'OwnerId', 'SalesforceOwnerName', 'XPOSvc',
       'Passthrough', 'FinalIncrease', 'FinalSuggested', 'Upload',
       'AutoQuote'],
      dtype='object')

In [23]:
test_grid.iloc[0,:]

Salesperson                     Barbara Rimmer-Rausch
Roll-up                                           RRD
Customer                 RRD Houston - Alpha Delta Pi
AcctNum                                          9539
Service                               Air First Class
Revenue Q1                                     2992.0
Revenue Q2                                     3015.0
Revenue Q3                                     3025.0
Margin Q1                                        0.35
Margin Q2                                        0.33
Margin Q3                                        0.37
Revenue (Q1-Q3)                                  9033
Margin (Q1-Q3)                                   0.35
Margin (Q2-Q3)                                   0.35
Rollup Margin (Q2-Q3)                             0.3
Suggested Increase                               0.06
Increase%                                        0.05
Pre-Call Notes                                    NaN
Call 1 Notes                

In [68]:
final_prep = test_grid.query('Upload == True')

In [71]:
final_prep.shape

(56, 41)

In [70]:
final_prep

,Salesperson,Roll-up,Customer,AcctNum,Service,Revenue Q1,Revenue Q2,Revenue Q3,Margin Q1,Margin Q2,...,SFProdId,AccountId,OwnerId,SalesforceOwnerName,XPOSvc,Passthrough,FinalIncrease,FinalSuggested,Upload,AutoQuote
0,Barbara Rimmer-Rausch,RRD,RRD Houston - Alpha Delta Pi,9539,Air First Class,2992.0,3015.0,3025.0,0.35,0.33,...,01t4x0000040fhyAAA,0014x00000H2Y6gAAF,0054x000004ga2fAAA,Barbara Rausch,1,False,0.050,0.060,True,True
1,Barbara Rimmer-Rausch,RRD,RRD Shakopee - US Bank,10195,02-IPA,0.0,0.0,2310.0,0.00,0.00,...,01t4x0000040fiAAAQ,0014x00001jYWSWAA4,0054x000004ga2fAAA,Barbara Rausch,19,True,0.000,0.000,True,True
2,Barbara Rimmer-Rausch,RRD,RRD-H&N Printing-Toastmasters Parcel,8561,03-Global Courier,5790.0,1753.0,360.0,0.47,0.42,...,01t4x0000040fiOAAQ,0014x00000H2W4cAAF,0054x000004ga2fAAA,Barbara Rausch,33,False,0.000,0.000,True,False
3,Barbara Rimmer-Rausch,RRD,RRD-H&N Printing-Toastmasters Parcel,8561,07-RRD Priority Parcel D/C DDU,200.0,0.0,0.0,0.86,0.00,...,01t4x0000040fjSAAQ,0014x00000H2W4cAAF,0054x000004ga2fAAA,Barbara Rausch,106,False,0.055,0.060,True,True
4,Barbara Rimmer-Rausch,RRD,RRD-H&N Printing-Toastmasters Parcel,8561,08-Global Courier,4033.0,19711.0,0.0,0.52,0.39,...,01t4x0000040fiPAAQ,0014x00000H2W4cAAF,0054x000004ga2fAAA,Barbara Rausch,33,False,0.000,0.000,True,False
5,Barbara Rimmer-Rausch,RRD,RRD-H&N Printing-Toastmasters Parcel,8561,09-RRD Priority Parcel DDU,4738.0,2496.0,0.0,0.58,0.44,...,01t4x0000040fjUAAQ,0014x00000H2W4cAAF,0054x000004ga2fAAA,Barbara Rausch,108,False,0.050,0.060,True,True
6,Barbara Rimmer-Rausch,RRD,RRD-Las Vegas-Amazon,9442,Canada Admail Std Machineable,0.0,9453.0,0.0,0.00,0.23,...,01t4x0000040fjCAAQ,0014x00000gKKsRAAW,0054x000004ga2fAAA,Barbara Rausch,86,False,0.020,0.020,True,True
7,Barbara Rimmer-Rausch,RRD,RRD-OCPC - Herbalife,9397,02-IPA,0.0,0.0,1791.0,0.00,0.00,...,01t4x0000040fiAAAQ,0014x00000H2Y6hAAF,0054x000004ga2fAAA,Barbara Rausch,19,False,0.000,0.000,True,False
8,Barbara Rimmer-Rausch,RRD,RRD-OCPC-Toastmasters,8746,07-RRD Priority Parcel D/C DDU,0.0,0.0,30.0,0.00,0.00,...,01t4x0000040fjSAAQ,0014x00000H2W4nAAF,0054x000004ga2fAAA,Barbara Rausch,106,False,0.050,0.060,True,True
9,Barbara Rimmer-Rausch,RRD,RRD-OCPC-Toastmasters,8746,Air Printed Material,302.0,0.0,41.0,0.22,0.00,...,01t4x0000040fhzAAA,0014x00000H2W4nAAF,0054x000004ga2fAAA,Barbara Rausch,3,False,0.050,0.065,True,True


## wtf int64 in places

In [72]:
increase_date = '2023-06-27'

In [73]:
#create a new opportunity, make one per acct
def format_opp_composite(opp_name, account_id, opp_index, owner_id, service_subdf):
    body = {
        'AccountId': account_id,
        'Name': opp_name,
        'Type': 'Existing Customer - Rate Increase',
        'StageName': 'Open',
        'CloseDate': increase_date,
        'Competitor__c': 'Other',
        'Amount': service_subdf['Revenue (Q1-Q3)'].astype(float).sum(),
        'OwnerId': owner_id
    }
    
    opp_post = {
        "method" : "POST",
        "url" : "/services/data/v53.0/sobjects/Opportunity/",
        "body" : body,
        "referenceId" : f'increase_opp__{opp_index}'
    }
    
    return [opp_post]

In [50]:
#create a new rate set, make one per acct
def format_ratesset_composite(opp_ref, account_id, ratesset_index, pickup, owner_id, description):
    body = {
        'Account__c': account_id,
        'Opportunity__c': f'@{{{opp_ref}.id}}',
        'Auto_Quote__c': True,
        'PickupPerLb__c': pickup,
        'Description__c': description,
        'OwnerId': owner_id
    }
    
    ratesset_post = {
        "method" : "POST",
        "url" : "/services/data/v53.0/sobjects/Rates_Set__c/",
        "body" : body,
        "referenceId" : f'increase_ratesset__{ratesset_index}'
    }
    
    return [ratesset_post]

In [75]:
final_prep.query('Upload').Status.unique()

array(['Pending Offer', 'Requote',
       'Requoted. Waiting on RRD finalization'], dtype=object)

In [84]:
print(final_prep['Status'].unique())


['Pending Offer' 'Requote' 'Requoted. Waiting on RRD finalization']


In [49]:
status_translator = {'Pending Offer':'Information Tentatively Confirmed', 'Requote':'Information Tentatively Confirmed', 
                     'Requoted. Waiting on RRD finalization':'Information Tentatively Confirmed'}

In [59]:
#create a quoted service for each line
def format_quoted_services_composite(ratesset_ref, opp_ref, services_subdf, owner_id, rateset_name):
    quotes_posts = []
    for i, service in services_subdf.iterrows():
        service_name = service['SalesforceProdName'].replace('/', '').replace(' DDPDDU', '')
        rates_name = f'{rateset_name} ({service_name})'
        if len(rates_name) > 80:
            n_truncate = len(rates_name) + 2 - 80
            end_index = len(rateset_name) - n_truncate
            rates_name = f'{rateset_name[:end_index]}...({service_name})'
        
        quote_margin = max(service['Margin (Q2-Q3)'], service['Rollup Margin (Q2-Q3)'])
        if service_name == '11-E-Packet':
            quote_margin = max(quote_margin, 0.15)
        quote_margin = service['RequoteMargin'] if service['RequoteMargin'] else quote_margin
        
        body = {
            'Rates_Set__c': f'@{{{ratesset_ref}.id}}',
            'Opportunity__c': f'@{{{opp_ref}.id}}',
            'Increase_Percent__c': round(service['FinalIncrease'] * 100, 2),
            'Increase_Percent_Suggested__c': round(service['FinalSuggested'] * 100, 2),
            'Product__c': service['SFProdId'],
            'Margin__c': round(quote_margin * 100, 2),
            'Increase_Margin_Q1__c': round(service['Margin Q1']* 100, 2),
            'Increase_Margin_Q2__c': round(service['Margin Q2']* 100, 2),
            'Increase_Margin_Q3__c': round(service['Margin Q3']* 100, 2),
            'Increase_Margin_Q1_3__c': round(service['Margin (Q1-Q3)']* 100, 2),
            'Increase_Margin_Q2_3__c': round(service['Margin (Q2-Q3)']* 100, 2),
            'Increase_Margin_Rollup__c': round(service['Rollup Margin (Q2-Q3)']* 100, 2),
            'Increase_Revenue_Q1__c': service['Revenue Q1'],
            'Increase_Revenue_Q2__c': service['Revenue Q2'],
            'Increase_Revenue_Q3__c': service['Revenue Q3'],
            'Increase_Revenue__c': float(service['Revenue (Q1-Q3)']),
            'PickupPerLb__c': service['Pu/lb'],
            'Postage_Passthrough__c': service['Passthrough'],
            'Name': rates_name,
            'OwnerId': owner_id,
            'StartDate__c': increase_date,
            'Auto_Quote__c': service['AutoQuote'],
            'Increase_Requote__c': service['Requote'] == 1,
            'Status__c': status_translator.get(service['Status'])
        }
        
        quote_post = {
            "method" : "POST",
            "url" : f"/services/data/v53.0/sobjects/Quoted_Service__c/",
            "body" : body,
            "referenceId" : f'increase_quoted_service__{i}'
            }
        quotes_posts.append(quote_post)
    
    return quotes_posts

## Now grouping by customer instead

In [32]:
graphs = []
i = 1
#connor_id = '0054x000004hEiaAAE'

for rollup, accounts_df in final_prep.groupby('Customer'):
    rep_count = accounts_df.Salesperson.nunique()
    for salesperson, sales_rollup_df in accounts_df.groupby('Salesperson'):
        first_line = sales_rollup_df.iloc[0,:]
        acct_name = first_line['Customer']
        accounts_count = sales_rollup_df.AccountId.nunique()
        if accounts_count > 1:
            opp_name = f'{rollup} 2023 Increase'
        else:
            opp_name = f'{acct_name} 2023 Increase'
        if rep_count > 1:
            opp_name += f' ({salesperson.split(" ")[0]})'
        owner_id = first_line['OwnerId']
        first_account_id = first_line['AccountId']
        #each graph is only one opportunity
        opp_composite = format_opp_composite(opp_name, first_account_id, 1, owner_id, sales_rollup_df)
        ratesset_composites = []
        quotes_composites = []
        for account_index, (account_id, account_services_df) in enumerate(sales_rollup_df.groupby('AccountId')):
            first_account_line = account_services_df.iloc[0, :]
            description = f'{first_account_line["AcctNum"]} {first_account_line["Customer"]} Rates'
            pickup = first_account_line['Pu/lb']
            ratesset_composite = format_ratesset_composite(opp_composite[0]['referenceId'], account_id, account_index, pickup, owner_id, description)
            ratesset_composites += ratesset_composite
            quotes_composites += format_quoted_services_composite(ratesset_composite[0]['referenceId'], opp_composite[0]['referenceId'], account_services_df, owner_id, description)
        graphs.append(
            {
                'graphId':i,
                 'compositeRequest': opp_composite + ratesset_composites + quotes_composites
            }
        )
        i += 1

request_body = {
        'graphs':graphs
    }

In [53]:
graphs

[{'graphId': 1,
  'compositeRequest': [{'method': 'POST',
    'url': '/services/data/v53.0/sobjects/Opportunity/',
    'body': {'AccountId': '0014x00000H2Y6gAAF',
     'Name': 'RRD Houston - Alpha Delta Pi 2023 Increase',
     'Type': 'Existing Customer - Rate Increase',
     'StageName': 'Open',
     'CloseDate': '2023-02-01',
     'Competitor__c': 'Other',
     'Amount': 9033.0,
     'OwnerId': '0054x000004ga2fAAA'},
    'referenceId': 'increase_opp__1'},
   {'method': 'POST',
    'url': '/services/data/v53.0/sobjects/Rates_Set__c/',
    'body': {'Account__c': '0014x00000H2Y6gAAF',
     'Opportunity__c': '@{increase_opp__1.id}',
     'Auto_Quote__c': True,
     'PickupPerLb__c': 0.12,
     'Description__c': '9539 RRD Houston - Alpha Delta Pi Rates',
     'OwnerId': '0054x000004ga2fAAA'},
    'referenceId': 'increase_ratesset__0'},
   {'method': 'POST',
    'url': '/services/data/v53.0/sobjects/Quoted_Service__c/',
    'body': {'Rates_Set__c': '@{increase_ratesset__0.id}',
     'Oppor

In [52]:
all_responses = []

In [54]:
for i in range(0,60,20):
    test_body = {'graphs': request_body['graphs'][i: i + 20]}
    res = sf_session.hit_endpoint(composite_graph_endpoint, test_body)
    with open(f"responses/rrd responses/responses thru {str(i).zfill(3)}-{str(i+20).zfill(3)}.json", "w") as f:
        f.write(res.text)
        all_responses.append(res.json())
    time.sleep(1)

In [55]:
expanded_responses = all_responses[0]['graphs']+all_responses[1]['graphs']+all_responses[2]['graphs']


In [56]:
expanded_responses[0]

{'graphId': '1',
 'graphResponse': {'compositeResponse': [{'body': [{'message': 'insufficient access rights on cross-reference id: 0014x00000H2Y6g',
      'errorCode': 'INSUFFICIENT_ACCESS_ON_CROSS_REFERENCE_ENTITY',
      'fields': []}],
    'httpHeaders': {},
    'httpStatusCode': 400,
    'referenceId': 'increase_opp__1'},
   {'body': [{'errorCode': 'PROCESSING_HALTED',
      'message': 'The transaction was rolled back since another operation in the same transaction failed.'}],
    'httpHeaders': {},
    'httpStatusCode': 400,
    'referenceId': 'increase_ratesset__0'},
   {'body': [{'errorCode': 'PROCESSING_HALTED',
      'message': 'The transaction was rolled back since another operation in the same transaction failed.'}],
    'httpHeaders': {},
    'httpStatusCode': 400,
    'referenceId': 'increase_quoted_service__0'}]},
 'isSuccessful': False}

In [57]:
 errors = {graph['graphId']: body for graph in expanded_responses for body in graph['graphResponse']['compositeResponse']}

In [58]:
errors

{'1': {'body': [{'errorCode': 'PROCESSING_HALTED',
    'message': 'The transaction was rolled back since another operation in the same transaction failed.'}],
  'httpHeaders': {},
  'httpStatusCode': 400,
  'referenceId': 'increase_quoted_service__0'},
 '2': {'body': [{'errorCode': 'PROCESSING_HALTED',
    'message': 'The transaction was rolled back since another operation in the same transaction failed.'}],
  'httpHeaders': {},
  'httpStatusCode': 400,
  'referenceId': 'increase_quoted_service__1'},
 '3': {'body': [{'errorCode': 'PROCESSING_HALTED',
    'message': 'The transaction was rolled back since another operation in the same transaction failed.'}],
  'httpHeaders': {},
  'httpStatusCode': 400,
  'referenceId': 'increase_quoted_service__31'},
 '4': {'body': [{'errorCode': 'PROCESSING_HALTED',
    'message': 'The transaction was rolled back since another operation in the same transaction failed.'}],
  'httpHeaders': {},
  'httpStatusCode': 400,
  'referenceId': 'increase_quoted_s